# Additional Skill: Q&A against a SQL Database (Azure SQL, Azure Fabric, Synapse, SQL Managed Instance, etc)

The goal of this notebook is to demonstrate how a LLM can understand a human question and translate that into a SQL query to get the answer. 

We will be using the Azure SQL Server that you created on the initial deployment. However the same code below works with any SQL database like Synapse for example.

Let's begin..

In [2]:
import os
import pandas as pd
import pyodbc
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.agent_toolkits import create_sql_agent, SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase

from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager

from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

from common.prompts import MSSQL_PROMPT, MSSQL_AGENT_PREFIX, MSSQL_AGENT_SUFFIX, MSSQL_AGENT_FORMAT_INSTRUCTIONS

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

In [3]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="ls__e98c2cdca9a74a9a9d533904c59aa81e"
os.environ["LANGCHAIN_PROJECT"]="skcc-aoai-demo"

# Install MS SQL DB driver in your machine


You might need the driver installed in order to talk to the SQL DB, so run the below cell once. Then restart the kernel and continue<br>
[Reference](https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline)

In [ ]:
# !sudo ./download_odbc_driver.sh

# Load Azure SQL DB with the Azure Billing CSV Data (will be updated)

The Azure SQL Database is currently empty, so we need to fill it up with data. Let's use the same data on the Covid CSV filed we used on the prior Notebook, that way we can compare results and methods. 
For this, you will need to type below the credentials you used at creation time.

In [4]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
import os

# Configuration for the database connection
db_config = {
    'drivername': 'mssql+pyodbc',
    'username': os.environ["SQL_SERVER_USERNAME"] + '@' + os.environ["SQL_SERVER_NAME"],
    'password': os.environ["SQL_SERVER_PASSWORD"],
    'host': os.environ["SQL_SERVER_NAME"],
    'port': 1433,
    'database': os.environ["SQL_SERVER_DATABASE"],
    'query': {'driver': 'ODBC Driver 17 for SQL Server'},
}

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection using the SQLAlchemy 2.0 execution style
with engine.connect() as conn:
    try:
        # Use the text() construct for safer SQL execution
        result = conn.execute(text("SELECT @@VERSION"))
        version = result.fetchone()
        print("Connection successful!")
        print(version)
    except Exception as e:
        print(e)


Connection successful!
('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tApr  3 2024 14:04:26 \n\tCopyright (C) 2022 Microsoft Corporation\n',)


# Query with LLM

**Note**: We are here using Azure SQL, however the same code will work with Synapse, SQL Managed instance, or any other SQL engine. You just need to provide the right values for the ENV variables and it will connect succesfully.

In [5]:
examples = [
    {
        "input": "Please tell me the total bill amount for jeju-utd.", 
        "query": "SELECT CompanyName, SUM(SumBillingPreTaxTotalPerMonth) AS TotalAmount FROM sampledata004_view_001 WHERE CompanyName = 'jeju-utd' GROUP BY CompanyName;"
    },
    {
        "input": "Please tell me the amount of each services for jeju-utd", 
        "query": "SELECT CompanyName, MeterCategory, SUM(SumBillingPreTaxTotalPerMonth) AS Amount FROM sampledata004_view_001 WHERE CompanyName = 'jeju-utd' GROUP BY CompanyName, MeterCategory;"
    }
]

In [ ]:
#MSSQL_SUFFIX = """I should look at the tables in the database to see what I can query. Then I should query the schema of the most relevant tables."""

In [6]:
MSSQL_SUFFIX = """
I should reference the below the schema of views.

CREATE VIEW [dbo].[sampledata004_view_001]
AS
SELECT  EntitlementDescription AS CompanyName, MeterCategory, ChargeStartDate, ChargeEndDate, UsageDate, SUM(BillingPreTaxTotal) AS SumBillingPreTaxTotalPerMonth
FROM    [dbo].[sampledata004]
GROUP BY EntitlementDescription, MeterCategory, ChargeStartDate, ChargeEndDate, UsageDate
GO

Column Description
- CompanyName : Company Name
- MeterCategory : Service Name
- ChargeStartDate : Charge Start Date
- ChargeEndDate : Charge End Date
- UsageDate : Usage Date
- BillingPreTaxTotal : Amount

"""

In [7]:
embedder = AzureOpenAIEmbeddings(deployment=os.environ["EMBEDDING_DEPLOYMENT_NAME"], chunk_size=1) 

In [8]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embedder,
    FAISS,
    k=5,
    input_keys=["input"],
)

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=MSSQL_PROMPT,
    suffix=MSSQL_SUFFIX,
)

full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [9]:
# Example formatted prompt
prompt_val = full_prompt.invoke(
    {
        "input": "Please tell me the total bill amount of kloudbank.",
        "top_k": 5,
        "dialect": "mssql",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct mssql query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't k

In [10]:
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=2000)

In [11]:
# Let's create the db object
db = SQLDatabase.from_uri(db_url)

In [12]:
db.run("select convert(varchar(25), getdate(), 120)")

"[('2024-05-03 08:25:45',)]"

### SQL Agent

LangChain has a SQL Agent which provides a more flexible way of interacting with SQL Databases than a chain. The main advantages of using the SQL Agent are:

    It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
    It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
    It can query the database as many times as needed to answer the user question.
    It will save tokens by only retrieving the schema from relevant tables.

To initialize the agent we’ll use the `create_sql_agent` constructor. This agent uses the SQLDatabaseToolkit which contains tools to:

    Create and execute queries
    Check query syntax
    Retrieve table descriptions
    … and more

In [13]:
agent_with_db = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)

In [14]:
# Natural Language question (query)
QUESTION = """please tell me the amount of each services for kloudbank during November,2023 in JSON format. please don't modify the result from database."""
#QUESTION = """please tell me the amount of each company during November,2023 in JSON format. please don't modify the result from database."""
#QUESTION = """please tell me the amount of Virtual Machines per company during November,2023 in JSON format. please don't modify the result from database."""
#QUESTION = """please tell me company lists having usage during November,2023 in JSON format. please don't modify the result from database."""
#QUESTION = """please tell me service lists having usage during November,2023 in JSON format. please don't modify the result from database."""

In [15]:
try:
    query_results = agent_with_db.invoke(QUESTION) 
except Exception as e:
    print(e)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `SELECT EntitlementDescription AS CompanyName, MeterCategory, SUM(BillingPreTaxTotal) AS Amount FROM sampledata004 WHERE EntitlementDescription = 'kloudbank' AND YEAR(UsageDate) = 2023 AND MONTH(UsageDate) = 11 GROUP BY EntitlementDescription, MeterCategory`


[('kloudbank', 'Application Gateway', 227775.45434570312), ('kloudbank', 'Automation', None), ('kloudbank', 'Azure DevOps', 134950.8137512207), ('kloudbank', 'Azure Grafana Service', 57748.626708984375), ('kloudbank', 'Azure Kubernetes Service', 141112.28649902344), ('kloudbank', 'Azure Monitor', 18173.12490338087), ('kloudbank', 'Bandwidth', 1333.259159493758), ('kloudbank', 'Container Registry', 63381.885681152344), ('kloudbank', 'Event Hubs', 12446.703186035156), ('kloudbank', 'Load Balancer', 127533.35648602247), ('kloudbank', 'Log Analytics', 159205.0537109375), ('kloudbank', 'Microsoft Defender for Cloud', 421483.69776821136), ('kloudbank', 'Service

In [15]:
print(query_results["output"])

```json
[
    {
        "CompanyName": "kloudbank",
        "MeterCategory": "Application Gateway",
        "Amount": 227775.45434570312
    },
    {
        "CompanyName": "kloudbank",
        "MeterCategory": "Automation",
        "Amount": null
    },
    {
        "CompanyName": "kloudbank",
        "MeterCategory": "Azure DevOps",
        "Amount": 134950.8137512207
    },
    {
        "CompanyName": "kloudbank",
        "MeterCategory": "Azure Grafana Service",
        "Amount": 57748.626708984375
    },
    {
        "CompanyName": "kloudbank",
        "MeterCategory": "Azure Kubernetes Service",
        "Amount": 141112.28649902344
    },
    {
        "CompanyName": "kloudbank",
        "MeterCategory": "Azure Monitor",
        "Amount": 18173.12490338087
    },
    {
        "CompanyName": "kloudbank",
        "MeterCategory": "Bandwidth",
        "Amount": 1333.259159493758
    },
    {
        "CompanyName": "kloudbank",
        "MeterCategory": "Container Registry",
     

In [16]:
template = """Answer the question thoroughly, based **ONLY** on the following context(It is JSON format):
{context}

If the question does not seem related to the context, just return "I don't know" as the answer.

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

llm2 = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=2000)

output_parser = StrOutputParser()

In [17]:
#FOLLOW_UP_QUESTION = """Which service have the highest billing costs? Plese let me know one service."""
#FOLLOW_UP_QUESTION = """Is the cost of Container Registry higher than Azure Kubernetes Service?"""
FOLLOW_UP_QUESTION = """Is the cost of Azure Kubernetes Service higher than Container Registry?"""

In [18]:
# Creation of our custom chain
chain = prompt | llm2 | output_parser

try:
    response = chain.invoke({"question": FOLLOW_UP_QUESTION, "context": query_results["output"]})
    print(response)
except Exception as e:
    print(e)

Yes, the cost of Azure Kubernetes Service is higher than Container Registry. The amount for Azure Kubernetes Service is 141112.28649902344, while the amount for Container Registry is 63381.885681152344. Therefore, the cost of Azure Kubernetes Service is significantly higher than Container Registry.


In [19]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_with_toolkit = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)

In [20]:
# As we know by now, Agents use expert/tools. Let's see which are the tools for this SQL Agent
agent_with_toolkit.tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f031182bb20>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f031182bb20>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f031182bb20>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [21]:
try:
    response = agent_with_toolkit.invoke(QUESTION) 
except Exception as e:
    response = str(e)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `SELECT CompanyName, MeterCategory, SUM(SumBillingPreTaxTotalPerMonth) AS Amount FROM sampledata004_view_001 WHERE CompanyName = 'kloudbank' AND MONTH(UsageDate) = 11 AND YEAR(UsageDate) = 2023 GROUP BY CompanyName, MeterCategory;`


[('kloudbank', 'Application Gateway', 227775.45434570312), ('kloudbank', 'Automation', None), ('kloudbank', 'Azure DevOps', 134950.8137512207), ('kloudbank', 'Azure Grafana Service', 57748.626708984375), ('kloudbank', 'Azure Kubernetes Service', 141112.28649902344), ('kloudbank', 'Azure Monitor', 18173.12490338087), ('kloudbank', 'Bandwidth', 1333.259159493758), ('kloudbank', 'Container Registry', 63381.885681152344), ('kloudbank', 'Event Hubs', 12446.703186035156), ('kloudbank', 'Load Balancer', 127533.35648602247), ('kloudbank', 'Log Analytics', 159205.0537109375), ('kloudbank', 'Microsoft Defender for Cloud', 421483.69776821136), ('kloudbank', 'Service Bus', 0.043909210595302284

In [ ]:
printmd(response["output"])

**IMPORTANT NOTE**: If you don't specify the column name on the question, runing the above cell multiple times will yield diferent results some times. <br>
The reason is:
The column names are ambiguous, hence it is hard even for Humans to discern what are the right columns to use

# Summary

In this notebook, we achieved our goal of Asking a Question in natural language to a dataset located on a SQL Database.  We did this by using purely prompt engineering (Langchain does it for us) and the cognitive power of GPT models.

This process shows why it is NOT necessary to move the data from its original source as long as the source has an API and a common language we can use to interface with. LLMs have been trained on the whole public Github corpus, so it can pretty much understand most of the coding and database query languages that exists out there. 